In [133]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import os
os.chdir('/Users/ltran/Documents/TrueData29/CPC_ML_tutorial/')

import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import random
import joypy
from matplotlib import cm
from datetime import datetime
from scipy import stats

from scipy.stats import fisher_exact
from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d
from sklearn.model_selection import train_test_split

# Set working directory
root_dir = '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

# Load TCA

In [2]:
data_dir = '/Users/ltran/Documents/Data/'


## Compute BMI

In [3]:
pheno = pd.read_csv(os.path.join(data_dir, 'TCA_vol/MRI_RDB_TCA_20221109_anonym.tsv'), sep = '\t')

In [4]:
# Pheno with relapses
pheno = pd.read_csv(os.path.join(data_dir, 'TCA_vol/MRI_RDB_TCA_20221207_anonym.tsv'), sep = '\t')

In [5]:
date_MRI = []
for i in pheno['MRIDateOfMRI']:
    if (type(i) == float):
        date_MRI.append(i)
    elif (type(i) == str):
        date_MRI.append(datetime.strptime(i, '%d/%m/%Y'))
        
pheno['date_MRI'] = date_MRI


pheno.loc[pheno.date_first_hospi == '02/22/2021', 'date_first_hospi'] = '22/02/2021'
date_hospi = []
for i in pheno['date_first_hospi']:
    if (type(i) == float):
        date_hospi.append(i)
    elif (type(i) == str):
        date_hospi.append(datetime.strptime(i, '%d/%m/%Y'))

pheno['date_hospi'] = date_hospi



In [6]:
time_delta = (pheno['date_MRI'] - pheno['date_hospi'])

In [7]:
tdl = []

for i in time_delta:
    tdl.append(float(i.days))
    
pheno['delta'] = tdl

# pheno.loc[(pheno['date_MRI'] < pheno['date_hospi']), 'delta'] = np.nan

In [8]:
# pheno = pheno.dropna(subset = 'delta')

In [9]:
# pheno['ASD'] = 'TCA'
# pheno.loc[pheno.delta > 44, 'ASD'] = 'q75_100'
# pheno.loc[pheno.delta < 44, 'ASD'] = 'q75'
# pheno.loc[pheno.delta < 14, 'ASD'] = 'q50'
# pheno.loc[pheno.delta < 7, 'ASD'] = 'q25'
# pheno.loc[pheno[pheno.TSA == 1].index, 'ASD'] = 'TCA_Autism'

In [10]:
pheno['ASD'] = 'TCA'
pheno.loc[pheno.delta < 14, 'ASD'] = 'TCA_ac'
pheno.loc[pheno.delta >= 14, 'ASD'] = 'TCA_pr'
# pheno.loc[pheno[pheno.TSA == 1].index, 'ASD'] = 'TCA_Autism'

In [11]:
pheno = pheno.dropna(subset = ['age_at_scan', 'Sex', 'machine'])

In [12]:
pheno = pheno.drop_duplicates(subset = ['subject_id'])

In [13]:
# Problème pour la taille (m -> cm)
pheno['size_at_scan'] = pheno['size_at_scan'].replace(0.92, 92)

In [14]:
# Rajouter suffixe à participant_id
pheno['participant_id'] = pheno['subject_id'].astype(str)+'_tca'

In [15]:
S = []
for i in pheno.subject_id:
    S.append('sub-'+str(i).zfill(4))
    
pheno['Subject'] = S

In [16]:
pheno.Subject = pheno.Subject+('_ses-0')+pheno.session_id.astype(str)

In [17]:
pheno['Protocole'] = pheno['sequence_3DT1'].str.upper()

## Remove failed

In [18]:
list_fail = [8, 12, 23, 18, 152, 26, 88, 73, 112, 194, 136]

In [19]:
pheno = pheno[~(pheno.subject_id.isin(list_fail))]

# Volume cortex

In [477]:
# cv_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/volumes-vlob-iso.txt'), sep = '\t')
# cv_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/volumes-vlob-tfe.txt'), sep = '\t')
# cv_iso.Subject = cv_iso.Subject.str.split('.', expand = True)[0]
# cv_tfe.Subject = cv_tfe.Subject.str.split('.', expand = True)[0]
# cv_iso['Protocole'] = 'ISO'
# cv_tfe['Protocole'] = 'TFE'
# cv = pd.concat([cv_iso, cv_tfe])
# cv['participant_id'] = cv.Subject.str.split('_', expand = True)[0].str.split('-', expand = True)[1].str.lstrip('0')
# cv['participant_id'] = cv['participant_id'].astype(str)+'_tca'

# Merge Volume, Area, CT

In [20]:
# Load ISO volume
vol_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/volumes-fs-iso.txt'), sep = '\t')

# Load TFE volume
vol_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/volumes-fs-tfe.txt'), sep = '\t')

In [21]:
vol_iso['Protocole'] = 'ISO'
vol_tfe['Protocole'] = 'TFE'

In [22]:
# Merge volume ISO + TFE 

vol_tca = pd.concat([vol_tfe, vol_iso])

In [23]:
lh_area_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.lh.area.csv'), sep = '\t') 
rh_area_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.rh.area.csv'), sep = '\t') 


lh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.lh.area.csv'), sep = '\t') 
rh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.rh.area.csv'), sep = '\t') 

area_iso = lh_area_iso.merge(rh_area_iso.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

area_tfe = lh_area_tfe.merge(rh_area_tfe.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

In [24]:
area_iso['Protocole'] = 'ISO'
area_tfe['Protocole'] = 'TFE'

In [25]:
area = pd.concat([area_iso, area_tfe])
area = area.rename(columns= {'lh.aparc.area' : 'Subject'})

In [26]:
lh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.lh.thickness.csv'), sep = '\t') 
rh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-iso.rh.thickness.csv'), sep = '\t') 

ct_iso = lh_ct_iso.merge(rh_ct_iso.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), 
               on = 'lh.aparc.thickness')


lh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.lh.thickness.csv'), sep = '\t') 
rh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TCA_vol/fs-7.2.0-tfe.rh.thickness.csv'), sep = '\t') 

ct_tfe = lh_ct_tfe.merge(rh_ct_tfe.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), 
               on = 'lh.aparc.thickness')

ct_iso['Protocole'] = 'ISO'
ct_tfe['Protocole'] = 'TFE'
ct = pd.concat([ct_iso, ct_tfe])
ct = ct.rename(columns= {'lh.aparc.thickness' : 'Subject'})

In [27]:
df_TCA = area.merge(vol_tca, on = ['Subject', 'Protocole']).merge(ct, on=['Subject','Protocole'])

In [28]:
l_pi = []
for i in df_TCA['Subject']:
    l_pi.append(int(i.split('-')[1].split('_')[0]))

df_TCA['participant_id'] = l_pi

In [29]:
df_TCA['participant_id'] = df_TCA['participant_id'].astype(str)+'_tca'

In [30]:
# df_TCA = df_TCA.merge(cv, on = ['participant_id', 'Protocole'])

In [31]:
df_TCA = df_TCA.merge(pheno, on = ['participant_id', 'Protocole'])

In [32]:
df_TCA['sex'] = df_TCA['Sex'].replace({'Female' : 2, 'Male' : 1})

In [33]:
# df_TCA = df_TCA.drop_duplicates(subset='participant_id')
df_TCA = df_TCA.dropna(subset='Sex')

In [34]:
df_TCA.columns = df_TCA.columns.str.replace('-', '_')

In [35]:
# df_TCA = df_TCA[~(df_TCA['magnetic_field_strength'] == '3')]

In [36]:
df_TCA = df_TCA.drop_duplicates(subset='participant_id', keep = 'first')

In [37]:
df_TCA['size_at_scan'] = df_TCA['size_at_scan'].replace(0.92, 92)

In [38]:
df_TCA['Month'] = df_TCA['age_at_scan']*12

In [39]:
df_TCA['scanner'] = df_TCA['manufacturer_model_name']

In [40]:
df_TCA = df_TCA.rename(columns= {'3rd_Ventricle' : 'third_Ventricle',
                       '4th_Ventricle' : 'fourth_Ventricle',
                        '5th_Ventricle' : 'fifth_Ventricle'})

### Export dataframe

In [41]:
df_TCA.to_csv(os.path.join(data_dir, 'Outputs/df_TCA.csv'), index = False)

In [42]:
df_TCA = df_TCA[~df_TCA.Subject_x.str.contains('069')]

In [43]:
df_TCA = df_TCA.rename(columns= {'weight_at_scan': 'patient_weight'})

In [44]:
df_TCA = df_TCA[(df_TCA.age_at_scan < 16) & (df_TCA.age_at_scan > 6)]

In [45]:
df_TCA = df_TCA[df_TCA.machine != 'Ingenia_3T']

In [46]:
df_TCA.to_csv(os.path.join(data_dir, 'Outputs/df_TCA.csv'))

In [47]:
df_TCA['ID'] = df_TCA.participant_id.str.split('_', expand = True)[0]

________________

# Load TSA

Load infos 

In [48]:
def is_duplicated(df, col):
    return df[df.duplicated(subset = col, keep = False)]
    

In [49]:
infos_TSA = pd.read_csv(os.path.join(data_dir, 'TSA_vol/MRI_RDB_20220928_anonym.tsv'), sep = '\t')

In [50]:
infos_TSA['Subject'] = 'sub-'+infos_TSA['subject_id'].astype(str).str.zfill(4)+'_ses-0'+infos_TSA['session_id'].astype(str
                                                                    )

In [51]:
# Enleve les NA dans sex, ASD, age
infos_TSA = infos_TSA.dropna(subset = ['Sex', 'age_at_scan', 'ASD'])
infos_TSA = infos_TSA[~(infos_TSA.ASD == '?')]

In [52]:
infos_TSA['sex'] = infos_TSA.Sex.replace({'Male' : 1, 'Female' : 2})

In [53]:
infos_TSA.loc[(infos_TSA.Group == 'Relative') & (infos_TSA.ASD == 'No'), 'ASD'] = 'Relative'

In [54]:
# maj_tsa = pd.read_csv(os.path.join(data_dir, 'TSA_vol/TSA_cohort_dcm_info_20221110_anonymized.csv'), sep = ',')

In [55]:
# maj_tsa = maj_tsa.rename(columns= {'SubCode' : 'subject_id'})

In [56]:
# infos_TSA = maj_tsa[['subject_id', 'patient_weight']].merge(infos_TSA,on = 'subject_id', suffixes = ('', '2'))

In [57]:
# infos_TSA = infos_TSA[(infos_TSA.age_at_scan > 6) & (infos_TSA.age_at_scan < 16)]

Load QC

In [58]:
qc_alex = pd.read_csv(os.path.join(data_dir, 'TSA_vol/Imagerie_QC.csv'), sep = ';')

In [59]:
# qc_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/qc-fs-6.0.0-tfe_20171002.txt'), sep = '\t')
# qc_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/qc-fs-6.0.0-isotrope_20171009.txt'), sep = '\t')
# iso_fail = qc_iso[qc_iso.QC <= 1][['Subject', 'QC']]
# tfe_fail = qc_tfe[qc_tfe.QC <= 1][['Subject', 'QC']]

In [60]:
QC = pd.read_csv(os.path.join(data_dir, 'QC_trio.csv'), sep = ';', on_bad_lines= 'skip')

In [61]:
QC.BIDS = QC.BIDS.str.lower()

In [62]:
QC = QC.dropna(subset = 'QC')
qc_alex = qc_alex.dropna(subset = 'QC')

In [63]:
QC = QC[~(QC.BIDS.isin(qc_alex.BIDS))]

In [64]:
QC = pd.concat([QC, qc_alex])

In [65]:
QC.session = QC.session.replace(np.nan, 1)

In [66]:
QC.loc[495, 'session'] = 2

In [67]:
# bids_f = QC[(QC.QC == 'F') | (QC.QC == 'g') | (QC.QC == 'M')][['BIDS', 'session', 'Protocole']]

In [68]:
bids_f = QC[(QC.QC == 'P')| (QC.QC == 'M')][['BIDS', 'session', 'Protocole', 'QC']]

In [69]:
ses = []
for i in bids_f['session']:
    if (i is np.nan):
        print(i)
    else:   
        ses.append(str(i).split('.')[0])
    
bids_f['ses'] = ses
bids_f = bids_f.replace({np.nan : '1'})

In [70]:
bids_f['Subject'] = bids_f['BIDS'].astype(str)+'_ses-0'+bids_f['ses'].astype(str) 

In [71]:
bids_f_iso = bids_f[(bids_f.Protocole == 'ISO') | (bids_f.Protocole == 'isotrope')
                    | (bids_f.Protocole == 'isot')]

In [72]:
bids_f_tfe  = bids_f[(bids_f.Protocole == 'TFE') | (bids_f.Protocole == 'tfe')]

In [73]:
bids_f_iso = bids_f_iso[~ (bids_f_iso.BIDS == 'sub-0361')]

Volumes Vlob

In [74]:
# cv_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/volumes-vlob-iso.txt'), sep = '\t')

# cv_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/volumes-vlob-tfe.txt'), sep = '\t')

# cv_iso.Subject = cv_iso.Subject.str.split('.', expand = True)[0]
# cv_tfe.Subject = cv_tfe.Subject.str.split('.', expand = True)[0]

# cv_iso['Protocole'] = 'ISO'
# cv_tfe['Protocole'] = 'TFE'

# cv_iso = cv_iso[~(cv_iso.Subject.isin(cv_tfe.Subject))]
# cv_iso = cv_iso[(cv_iso.Subject.isin(bids_f_iso.Subject))]
# cv_tfe = cv_tfe[(cv_tfe.Subject.isin(bids_f_tfe.Subject))]
# cv = pd.concat([cv_iso, cv_tfe])

In [75]:
# cv['participant_id'] = cv.Subject.str.split('_', expand = True)[0].str.split('-', expand = True)[1].str.lstrip('0')
# cv['participant_id'] = cv['participant_id'].astype(int)

Volumes

In [76]:
# Load ISO volume
vol_tsa_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.aseg.csv'), sep = '\t')

# Load TFE volume
vol_tsa_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.aseg.csv'), sep = '\t')

In [77]:
vol_tsa_iso = vol_tsa_iso.rename(columns= {'Measure:volume':'Subject'})
vol_tsa_tfe = vol_tsa_tfe.rename(columns= {'Measure:volume':'Subject'})

In [78]:
vol_tsa_iso = vol_tsa_iso[(vol_tsa_iso.Subject.isin(bids_f_iso.Subject))]

In [79]:
# vol_tsa_iso = vol_tsa_iso.merge(bids_f_iso, on = 'Subject')
# vol_tsa_tfe = vol_tsa_tfe.merge(bids_f_tfe, on = 'Subject')

In [80]:
vol_tsa_tfe = vol_tsa_tfe[(vol_tsa_tfe.Subject.isin(bids_f_tfe.Subject))]

In [81]:
vol_tsa_iso= vol_tsa_iso[~(vol_tsa_iso.Subject.isin(vol_tsa_tfe.Subject))]

In [82]:
vol_tsa_iso['Protocole'] = 'ISO'
vol_tsa_tfe['Protocole'] = 'TFE'

In [83]:
vol_tsa = pd.concat([vol_tsa_iso, vol_tsa_tfe])

In [84]:
participants = []
for i in vol_tsa['Subject']:
    participants.append(int(i.split('-')[1].split('_')[0]))
    
vol_tsa['participant_id'] = participants

In [85]:
vol_tsa = vol_tsa.drop_duplicates(subset= 'Subject', keep = 'first')

Surface area

In [86]:
rh_area_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.rh.area.csv'), sep = '\t')
lh_area_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.lh.area.csv'), sep = '\t')
rh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.rh.area.csv'), sep = '\t')
lh_area_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.lh.area.csv'), sep = '\t')

In [87]:
area_iso = lh_area_iso.merge(rh_area_iso.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

In [88]:
area_tfe = lh_area_tfe.merge(rh_area_tfe.rename(columns={'rh.aparc.area': 'lh.aparc.area'}), on ='lh.aparc.area')

In [89]:
area_iso = area_iso.rename(columns= {'lh.aparc.area' : 'Subject'})
area_tfe = area_tfe.rename(columns= {'lh.aparc.area' : 'Subject'})

In [90]:
area_iso = area_iso[(area_iso.Subject.isin(bids_f_iso.Subject))]

In [91]:
area_tfe = area_tfe[(area_tfe.Subject.isin(bids_f_tfe.Subject))]

In [92]:
area_iso= area_iso[~(area_iso.Subject.isin(area_tfe.Subject))]

In [93]:
area_iso['Protocole'] = 'ISO'
area_tfe['Protocole'] = 'TFE'

In [94]:
area_tsa = pd.concat([area_iso, area_tfe])

In [95]:
participants = []
for i in area_tsa['Subject']:
    participants.append(int(i.split('-')[1].split('_')[0]))
    
area_tsa['participant_id'] = participants

In [96]:
area_tsa = area_tsa.drop_duplicates(subset= 'Subject', keep = 'first')

Cortical thickness

In [97]:
rh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.rh.thickness.csv'), sep = '\t')
lh_ct_iso = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-iso.lh.thickness.csv'), sep = '\t')
rh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.rh.thickness.csv'), sep = '\t')
lh_ct_tfe = pd.read_csv(os.path.join(data_dir, 'TSA_vol/fs-7.2.0-tfe.lh.thickness.csv'), sep = '\t')

In [98]:
ct_iso = lh_ct_iso.merge(rh_ct_iso.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), on ='lh.aparc.thickness')

In [99]:
ct_tfe = lh_ct_tfe.merge(rh_ct_tfe.rename(columns={'rh.aparc.thickness': 'lh.aparc.thickness'}), on ='lh.aparc.thickness')

In [100]:
ct_iso = ct_iso.rename(columns={'lh.aparc.thickness':'Subject'})
ct_tfe = ct_tfe.rename(columns={'lh.aparc.thickness':'Subject'})

In [101]:
ct_iso = ct_iso[(ct_iso.Subject.isin(bids_f_iso.Subject))]

In [102]:
ct_tfe = ct_tfe[(ct_tfe.Subject.isin(bids_f_tfe.Subject))]

In [103]:
ct_iso= ct_iso[~(ct_iso.Subject.isin(ct_tfe.Subject))]

In [104]:
ct_iso['Protocole'] = 'ISO'
ct_tfe['Protocole'] = 'TFE'

In [105]:
ct_tsa = pd.concat([ct_iso, ct_tfe])

In [106]:
participants = []
for i in ct_tsa['Subject']:
    participants.append(int(i.split('-')[1].split('_')[0]))
    
ct_tsa['participant_id'] = participants

In [107]:
ct_tsa = ct_tsa.drop_duplicates(subset= 'Subject', keep = 'first')

### Merge volume / area / ct

In [108]:
df_tsa = area_tsa.merge(ct_tsa, on = ['Subject', 'Protocole'], suffixes=('', '_y')).merge(vol_tsa, on = ['Subject','Protocole'], suffixes=('', '_z'))

In [109]:
# df_tsa = df_tsa.merge(cv, on = ['Subject', 'Protocole'])

In [110]:
# df_tsa = area_tsa.merge(ct_tsa, on = 'participant_id', suffixes=('', '_y')).merge(vol_tsa, on = 'participant_id', suffixes=('', '_z'))

### Merge with infos_TSA

In [111]:
zf = []
for i in infos_TSA.subject_id:
    zf.append('sub-'+str(i).zfill(4)+'_ses-0')
    
infos_TSA['Subject'] = zf+infos_TSA.session_id.astype(str)

In [112]:
# df_TSA = infos_TSA.merge(cv, on = 'Subject')

In [113]:
df_TSA = infos_TSA.merge(df_tsa,on = 'Subject')

In [114]:
df_TSA['participant_id'] = df_TSA.Subject.str.split('-', expand = True)[1].str.split('_', expand = True)[0].str.lstrip('0')

In [115]:
df_TSA.columns = df_TSA.columns.str.replace('-', '_')

In [116]:
df_TSA['scanner'] = df_TSA['machine'].str.upper()

In [117]:
df_TSA = df_TSA.rename(columns= {'3rd_Ventricle' : 'third_Ventricle',
                       '4th_Ventricle' : 'fourth_Ventricle',
                        '5th_Ventricle' : 'fifth_Ventricle'})

## ASD Categories

In [118]:
df_TSA.loc[(df_TSA.ASD == 'Yes') & (df_TSA.Group !=  'Patient'), 'ASD'] = 'Pas_ASD'

In [119]:
df_TSA.loc[(df_TSA.ASD == 'No') & (df_TSA.Group ==  'Patient'), 'ASD'] = 'TD_Patient'

## Export TSA

In [120]:
df_TSA.to_csv(os.path.join(data_dir, 'Outputs/df_TSA.csv'), index = False)

# Merge TCA + TSA

In [121]:
df_TSA = df_TSA.drop(columns= 'participant_id_y')

In [122]:
df_tsa_tca = pd.concat([df_TSA, df_TCA])

### Clean NA

In [123]:
df_tsa_tca = df_tsa_tca.dropna(subset=['Sex', 'age_at_scan', 'ASD', 'scanner'])

### Filter 1.5 T / 3 T


In [124]:
df_tsa_tca = df_tsa_tca[df_tsa_tca.scanner != 'INGENIA_3T']

In [125]:
df_tsa_tca = df_tsa_tca[~((df_tsa_tca['magnetic_field_strength'] == '3')|(df_tsa_tca['machine'] == 'Ingenia_3T'))]

### Filter on age

In [126]:
df_tsa_tca = df_tsa_tca[(df_tsa_tca['age_at_scan'] > 6) & (df_tsa_tca['age_at_scan'] < 16)]

In [127]:
df_tsa_tca = df_tsa_tca.drop_duplicates('participant_id')

In [128]:
df_tsa_tca.groupby(['Sex', 'ASD']).size().to_frame('Count')

Count
Sex    ASD              
Female No             37
       Pas_ASD         5
       Relative       16
       TCA             3
       TCA_ac         34
       TCA_pr         43
       TD_Patient      1
       Yes            49
Male   No             49
       Pas_ASD         3
       Relative       14
       TCA             1
       TCA_ac          6
       TCA_pr          6
       Yes           240

## Export TSA + TCA

In [ ]:
thick = [col for col in df_tsa_tca.columns if '_thickness' in col]
thick.remove('lh_MeanThickness_thickness')
thick.remove('rh_MeanThickness_thickness')
len(thick)

df_tsa_tca['meanCT'] = df_tsa_tca[['lh_MeanThickness_thickness', 'rh_MeanThickness_thickness']].mean(axis=1)

In [588]:
arean  = [col for col in df_tsa_tca.columns if '_area' in col]
arean.remove('rh_WhiteSurfArea_area')
arean.remove('lh_WhiteSurfArea_area')
arean.remove('Left_Accumbens_area'),
arean.remove('Right_Accumbens_area')

df_tsa_tca['totalSA'] = df_tsa_tca[arean].sum(axis = 1)

In [589]:
df_tsa_tca.to_csv(os.path.join(data_dir, 'Outputs/df_tsa_tca.csv'))

(Optional) Add ARFID diag

In [386]:
df_tsa_tca.loc[df_tsa_tca.principal_diag == 2, 'ASD'] = 'T_ARFID'

In [ ]:
df_tsa_tca.to_csv(os.path.join(data_dir, 'Outputs/df_tsa_tca.csv'))